[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/accdavlo/excellence-MOR/blob/main/codes/heat_cahn_hillard_FD.ipynb)

# Heat equation

In this notebook we will solve the 1D Heat parabolic equation given 
$$\partial_t u -\partial_{xx} u = f .$$

Let us start with an example on the domain $\Omega=[0,1]$  and time domain $[0,T]$ with $T=1$ with $f=0$, homogeneous Dirichlet BC $u(0)=u(2\pi)=0$ and initial condition $u_0(x)=\sin(x)$.

In [ ]:
import numpy as np               # Library for numerical computations
import matplotlib.pyplot as plt  # Library for plotting
import scipy.sparse as sp           # Library for sparse matrices

In [ ]:
class Geometry1D:
    """1D geometry class for finite difference discretization.
    
    Attributes:
        x_left (float): Left boundary of the domain
        x_right (float): Right boundary of the domain
        N (int): Number of grid points
        xx (ndarray): Array of grid points
        dx (float): Grid spacing
    """
    def __init__(self, x_left, x_right, N=None):
        """Initialize 1D geometry.
        
        Args:
            x_left (float): Left boundary coordinate
            x_right (float): Right boundary coordinate
            N (int, optional): Number of grid points. If provided, creates uniform grid.
        """
        self.x_left = x_left
        self.x_right = x_right
        if N is not None:
            self.set_N(N)
    
    def set_linspace(self, xx):
        """Set grid points from an array.
        
        Args:
            xx (ndarray): Array of grid points
        """
        self.xx = xx
        self.x_left = self.xx[0]
        self.x_right = self.xx[-1]
        self.N = len(self.xx)
    
    def set_N(self, N):
        """Create uniform grid with N points.
        
        Args:
            N (int): Number of grid points
        """
        self.N = N
        self.xx = np.linspace(self.x_left, self.x_right, N)
        self.dx = self.xx[1] - self.xx[0]

In [ ]:
# Test geometry and initial condition
geom = Geometry1D(0,2*np.pi,100)
u0 = np.sin(geom.xx)
T_end = 1.
plt.plot(geom.xx,u0)

u_ex = lambda t,x: np.exp(-t)*np.sin(x)

### Implicit Euler

$$
\frac{u^{n}_i-u^{n-1}_i}{\Delta t} - \frac{u_{i+1}^n-2u_i^n+u_{i-1}^n}{\Delta x^2}=0 
\Longleftrightarrow u^{n}_i-\frac{\Delta t}{\Delta x^2}(u_{i+1}^n-2u_i^n+u_{i-1}^n) = u^{n-1}_i
$$

### Linear systems with matrix
$$
\begin{pmatrix}
1+2\frac{\Delta t}{\Delta x^2} &-\frac{\Delta t}{\Delta x^2} & 0&\dots & \dots\\
-\frac{\Delta t}{\Delta x^2} &1+2\frac{\Delta t}{\Delta x^2} &-\frac{\Delta t}{\Delta x^2} &\dots & \dots\\
\vdots & \ddots & \ddots & \ddots &\vdots\\
0&\dots & \dots &-\frac{\Delta t}{\Delta x^2} &1+2\frac{\Delta t}{\Delta x^2}     
\end{pmatrix}
$$


In [ ]:
Nx = 10
dt = 0.01
geom = Geometry1D(0,2*np.pi,Nx)
u0_lambda = lambda x: np.sin(x)
T_end = 1.


# This time we need to solve a linear system, so we have to assemble some matrices
# Assemble the second derivative matrix D2: (D2 @ u)_i= u_{i+1}-2u_i + u_{i-1} for every i (when possible)
# Note that it is a tridiagonal matrix! So we can use the np.diag function to get its discretization.
# P.S. For the moment DO NOT INCLUDE the term 1/dx^2
def assemble_deriv2_matrix(geom):
    deriv2_matrix = sp.diags([-2*np.ones(geom.N), np.ones(geom.N-1), np.ones(geom.N-1)], [0, 1, -1], format='csr')
    return deriv2_matrix

## Define the second derivative matrix
deriv2_matrix = assemble_deriv2_matrix(geom)


def put_zero_row_in_csr(A, i):
    A.data[A.indptr[i]:A.indptr[i+1]] = 0

print(deriv2_matrix.toarray())
put_zero_row_in_csr(deriv2_matrix,0)

print(" ")
print(deriv2_matrix.toarray())
deriv2_matrix[0,0]=1.

print(" ")
print(deriv2_matrix.toarray())

In [ ]:
Nx = 50
dt = 0.01
geom = Geometry1D(0,2*np.pi,Nx)
u0_lambda = lambda x: np.sin(x)
T_end = 1.


# This time we need to solve a linear system, so we have to assemble some matrices
# Assemble the second derivative matrix D2: (D2 @ u)_i= u_{i+1}-2u_i + u_{i-1} for every i (when possible)
# Note that it is a tridiagonal matrix! So we can use the np.diag function to get its discretization.
# P.S. For the moment DO NOT INCLUDE the term 1/dx^2
def assemble_deriv2_matrix(geom):
    deriv2_matrix = sp.diags([-2*np.ones(geom.N), np.ones(geom.N-1), np.ones(geom.N-1)], [0, 1, -1], format='csr')
    return deriv2_matrix



## Define the second derivative matrix
deriv2_matrix = assemble_deriv2_matrix(geom)

## Define the lhs matrix as I-dt/dx^2 D2
lhs_matrix = sp.eye(geom.N) - dt/geom.dx**2*deriv2_matrix


def put_zero_row_in_csr(A, i):
    A.data[A.indptr[i]:A.indptr[i+1]] = 0

## Boundary conditions of the lhs matrix puts everything to zero on boundary lines, except the diagonal term to 1
def apply_BC_matrix(lhs_matrix, geom):
    put_zero_row_in_csr(lhs_matrix, 0)
    lhs_matrix[0,0] = 1.
    put_zero_row_in_csr(lhs_matrix, geom.N-1)
    lhs_matrix[-1,-1] = 1.

apply_BC_matrix(lhs_matrix, geom)


# On the RHS vector we will put the values of the exact solution
def apply_BC_vector(rhs, uL = 0., uR = 0.):
    rhs[0] = uL
    rhs[-1] = uR



class Heat_implicit_euler:
    def __init__(self, geom, T_end, u0_lambda, dt, dt_save = 0.1, u_ex_lambda = None):
        self.geom = geom
        self.T_end = T_end
        self.u0_lambda = u0_lambda
        self.u0 = self.u0_lambda(self.geom.xx)
        self.set_dt(dt)
        self.dt_save = dt_save
        self.Nt_save = np.int64(self.T_end//self.dt_save +2)
        if u_ex_lambda is not None:
            self.u_ex_lambda = u_ex_lambda
        self.assemble_lhs()

    def assemble_lhs(self):
        ## Assemble the left hand side as I-dt/dx^2 D2 and apply BC
        self.deriv2_matrix = sp.diags([-2*np.ones(self.geom.N), np.ones(self.geom.N-1), np.ones(self.geom.N-1)], [0, 1, -1], format='csr')
        self.lhs_matrix = sp.eye(self.geom.N) - self.dt/self.geom.dx**2*self.deriv2_matrix
        apply_BC_matrix(self.lhs_matrix, self.geom)
    
    def assemble_rhs(self,un):
        # Assemble rhs and apply BC
        rhs = un
        apply_BC_vector(rhs)
        return rhs

    def set_geom(self,dx=None,Nx=None):
        # Differently from explicit, when we change the points, we have to reassemble the lhs 
        if Nx is None:
            Nx = np.int64((self.geom.x_right-self.geom.x_left)/dx)+1
        self.geom.set_N(Nx)
        self.u0 = self.u0_lambda(self.geom.xx)
        self.assemble_lhs() 
        
    def set_dt(self,dt):
        # Differently from explicit, when we change the timestep, we have to reassemble the lhs 
        self.dt = dt
        self.Nt = np.int64(self.T_end//self.dt+2)        
        self.assemble_lhs()
        
    def evolve(self):

        self.U_sol=np.zeros((self.Nt_save,self.geom.N))
        self.U_sol[0] = self.u0
        un = np.copy(self.u0)
        un1 = np.copy(self.u0)

        it=0
        it_save = 0
        time = 0.
        time_save = 0.
        self.times = [time]
        while ( it<self.Nt and time<self.T_end):

            time=time+self.dt
            time_save = time_save+self.dt
            it+=1
            
            # Assemble rhs (the lhs is already assembled)
            rhs = self.assemble_rhs(un)
            
            # Solve the linear system
            un1 = sp.linalg.spsolve(self.lhs_matrix,rhs)

            un = un1

            if time_save>self.dt_save:
                it_save +=1
                self.U_sol[it_save,:] = un1
                self.times.append(time)
                time_save = 0.

        if hasattr(self,"u_ex_lambda"):
            # Final time error
            self.u_ex_end = self.u_ex_lambda(time,self.geom.xx)
            self.error = np.linalg.norm(un1-self.u_ex_end)/np.linalg.norm(self.u_ex_end)
            print("Final error ",self.error)
        return self.geom.xx, un1
    
IE_FD_approx = Heat_implicit_euler(geom, T_end, u0_lambda, dt, dt_save = 0.1, u_ex_lambda = u_ex)
xx, un1 = IE_FD_approx.evolve()

fig,(ax1,ax2) = plt.subplots(1,2,figsize=(10,5))
for it, time in enumerate(IE_FD_approx.times):
    ax1.plot(geom.xx, IE_FD_approx.U_sol[it], label="Time = %1.3f"%time)

ax2.plot(geom.xx, IE_FD_approx.u_ex_end, label="Exact final time")
ax2.plot(geom.xx, un1, label="Approx final time")
ax1.legend()
ax2.legend()

fig.suptitle(r"Implicit Euler $\Delta x=%1.4f, \, \Delta t=%1.4f$"%(geom.dx,IE_FD_approx.dt))
plt.show()




In [ ]:
IE_FD_approx.set_geom(dx = 0.01)
IE_FD_approx.set_dt(0.01)
xx, un1 = IE_FD_approx.evolve()

fig,(ax1,ax2) = plt.subplots(1,2,figsize=(10,5))
for it, time in enumerate(IE_FD_approx.times):
    ax1.plot(geom.xx, IE_FD_approx.U_sol[it], label="Time = %1.3f"%time)

ax2.plot(geom.xx, IE_FD_approx.u_ex_end, label="Exact final time")
ax2.plot(geom.xx, un1, label="Approx final time")
ax1.legend()
ax2.legend()

fig.suptitle(r"Implicit Euler $\Delta x=%1.4f, \, \Delta t=%1.4f$"%(geom.dx,IE_FD_approx.dt))
plt.show()


### Various Initial Contitions

In [ ]:
# Various frequencies wave

u0_lambda = lambda x: np.sin(x)+0.8*np.sin(5.*x)
u_ex = lambda t,x: np.exp(-t)*np.sin(x)+0.8*np.exp(-t*25)*np.sin(5*x)
geom = Geometry1D(0.,2*np.pi, 100)
T_end = 0.5
IE_FD_approx = Heat_implicit_euler(geom, T_end, u0_lambda, dt=0.01, dt_save = 0.1, u_ex_lambda = u_ex)
geom = IE_FD_approx.geom
xx, un1 = IE_FD_approx.evolve()

fig,(ax1,ax2) = plt.subplots(1,2,figsize=(10,5))
for it, time in enumerate(IE_FD_approx.times):
    ax1.plot(geom.xx, IE_FD_approx.U_sol[it], label="Time = %1.3f"%time)

ax2.plot(geom.xx, IE_FD_approx.u_ex_end, label="Exact final time")
ax2.plot(geom.xx, un1, label="Approx final time")
ax1.legend()
ax2.legend()

fig.suptitle(r"Implicit Euler $\Delta x=%1.4f, \, \Delta t=%1.4f$"%(geom.dx,IE_FD_approx.dt))
plt.show()


In [ ]:
# Discontinuous IC
# Try different dt!!

u0_lambda = lambda x: (x<0.75)*(x>0.25)
geom = Geometry1D(0.,1.0, 100)
T_end = 0.5

IE_FD_approx = Heat_implicit_euler(geom, T_end, u0_lambda, dt=0.01, dt_save = 0.1)
geom = IE_FD_approx.geom
xx, un1 = IE_FD_approx.evolve()

for it, time in enumerate(IE_FD_approx.times):
    plt.plot(geom.xx, IE_FD_approx.U_sol[it], label="Time = %1.3f"%time)
plt.title("Implicit Euler")
plt.legend()
plt.show()


# Nonlinear problem (Cahn-Hilliard)

$$
\partial_t u -\partial_{xx} u  = u-u^3
$$

In [ ]:

class Cahn_Hillard_implicit_euler:
    def __init__(self, geom, T_end, u0_lambda, dt, dt_save = 0.1, max_newton_iter = 10, diffusion=1.,u_ex_lambda = None):
        self.geom = geom
        self.T_end = T_end
        self.diffusion = diffusion
        self.u0_lambda = u0_lambda
        self.u0 = self.u0_lambda(self.geom.xx)
        self.set_dt(dt)
        self.max_newton_iter = max_newton_iter
        self.dt_save = dt_save
        self.Nt_save = np.int64(self.T_end//self.dt_save +2)
        if u_ex_lambda is not None:
            self.u_ex_lambda = u_ex_lambda
        self.assemble_matrices()

    def assemble_matrices(self):
        ## Assemble the left hand side as I-dt/dx^2 D2 and apply BC
        self.deriv2_matrix = -sp.diags(2.*np.ones(self.geom.N), 0) + sp.diags(np.ones(self.geom.N-1), 1) + sp.diags(np.ones(self.geom.N-1), -1)
        self.lhs_matrix = sp.eye(self.geom.N) - self.dt/self.geom.dx**2*self.diffusion*self.deriv2_matrix
        apply_BC_matrix(self.lhs_matrix, self.geom)

    def assemble_jacobian(self, u):
        self.jacobian = sp.eye(self.geom.N) - self.dt/self.geom.dx**2*self.diffusion*self.deriv2_matrix - self.dt*sp.diags([3*u**2-1],[0])
        apply_BC_matrix(self.jacobian, self.geom)

    def assemble_residual(self,u,un):
        F = u - un - self.dt*(self.diffusion*self.deriv2_matrix/self.geom.dx**2 @ u + u - u**3)
        apply_BC_vector(F, uL = 0., uR = 0.)
        return F
    def assemble_rhs(self,un):
        # Assemble rhs and apply BC
        rhs = un
        apply_BC_vector(rhs)
        return rhs

    def set_geom(self,dx=None,Nx=None):
        # Differently from explicit, when we change the points, we have to reassemble the lhs 
        if Nx is None:
            Nx = np.int64((self.geom.x_right-self.geom.x_left)/dx)+1
        self.geom.set_N(Nx)
        self.u0 = self.u0_lambda(self.geom.xx)
        self.assemble_matrices() 
        
    def set_dt(self,dt):
        # Differently from explicit, when we change the timestep, we have to reassemble the lhs 
        self.dt = dt
        self.Nt = np.int64(self.T_end//self.dt+2)        
        self.assemble_matrices()
        
    def evolve(self):

        self.U_sol=np.zeros((self.Nt_save,self.geom.N))
        self.U_sol[0] = self.u0
        un = np.copy(self.u0)
        un1 = np.copy(self.u0)

        it=0
        it_save = 0
        time = 0.
        time_save = 0.
        self.times = [time]
        while ( it<self.Nt and time<self.T_end):

            time=time+self.dt
            time_save = time_save+self.dt
            it+=1
            
            # Assemble rhs (the lhs is already assembled)
            rhs = self.assemble_rhs(un)
            
            # Solve the nonlinear system with newton's method
            u_iter = un.copy()
            for it_newton in range(self.max_newton_iter):
                self.assemble_jacobian(u_iter)

                F  = self.assemble_residual(u_iter,un)

                delta = sp.linalg.spsolve(self.jacobian,-F)
                apply_BC_vector(delta, uL = 0., uR = 0.)

                u_iter += delta
                err_delta = np.linalg.norm(delta)/np.sqrt(self.geom.N)
                err_F = np.linalg.norm(F)/np.sqrt(self.geom.N)
                if  (err_delta<1e-10) or (err_F<1e-10):
                    #print("Newton converged in %d iterations"%(it_newton))
                    break
            
            un1 = u_iter

            un = un1

            if time_save>self.dt_save:
                it_save +=1
                self.U_sol[it_save,:] = un1
                self.times.append(time)
                time_save = 0.

        if hasattr(self,"u_ex_lambda"):
            # Final time error
            self.u_ex_end = self.u_ex_lambda(time,self.geom.xx)
            self.error = np.linalg.norm(un1-self.u_ex_end)/np.linalg.norm(self.u_ex_end)
            print("Final error ",self.error)
        return self.geom.xx, un1
    
    
Nx = 200
dt = 0.01
geom = Geometry1D(0,2*np.pi,Nx)

T_end = 1.
diffusion = 0.01  #Play with the diffusion parameter to see how the solution changes
u0_lambda = lambda x: np.sin(x)**3

IE_FD_approx = Cahn_Hillard_implicit_euler(geom, T_end, u0_lambda, dt, diffusion=diffusion,dt_save = 0.1)
xx, un1 = IE_FD_approx.evolve()

fig,(ax1,ax2) = plt.subplots(1,2,figsize=(10,5))
for it, time in enumerate(IE_FD_approx.times):
    ax1.plot(geom.xx, IE_FD_approx.U_sol[it], label="Time = %1.3f"%time)

ax2.plot(geom.xx, un1, label="Approx final time")
ax1.legend()
ax2.legend()

fig.suptitle(r"Implicit Euler $\Delta x=%1.4f, \, \Delta t=%1.4f$"%(geom.dx,IE_FD_approx.dt))
plt.show()


